<a href="https://colab.research.google.com/github/2303A52069/Generative_Ai/blob/main/2303A52069_GENAI_A8_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras import regularizers


In [2]:
data = pd.read_csv('/content/winequality-red.csv')


In [3]:
data.head()


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [4]:
X = data.drop(columns=['quality'])
y = data['quality'].values.reshape(-1, 1)

In [5]:
data['quality'].value_counts()

,count
quality,
5,681
6,638
7,199
4,53
8,18
3,10


In [6]:
encoder = OneHotEncoder(sparse_output=False)
y = encoder.fit_transform(y)

In [7]:
# y = (y >= 6).astype(int)

In [8]:
y

array([[0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       ...,
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.]])

In [9]:
X_train, X_test1, y_train, y_test1 = train_test_split(X, y, test_size=0.3, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test1, y_test1, test_size=0.5, random_state=42)

In [10]:
X_test

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
629,7.6,0.685,0.23,2.3,0.111,20.0,84.0,0.99640,3.21,0.61,9.3
100,8.3,0.610,0.30,2.1,0.084,11.0,50.0,0.99720,3.40,0.61,10.2
1094,6.6,0.725,0.09,5.5,0.117,9.0,17.0,0.99655,3.35,0.49,10.8
350,10.7,0.670,0.22,2.7,0.107,17.0,34.0,1.00040,3.28,0.98,9.9
1488,5.6,0.540,0.04,1.7,0.049,5.0,13.0,0.99420,3.72,0.58,11.4
...,...,...,...,...,...,...,...,...,...,...,...
464,11.5,0.315,0.54,2.1,0.084,5.0,15.0,0.99870,2.98,0.70,9.2
1240,7.5,0.610,0.20,1.7,0.076,36.0,60.0,0.99494,3.10,0.40,9.3
1265,7.2,0.570,0.05,2.3,0.081,16.0,36.0,0.99564,3.38,0.60,10.3
285,9.9,0.590,0.07,3.4,0.102,32.0,71.0,1.00015,3.31,0.71,9.8


In [11]:
y_test

array([[0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       ...,
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.]])

In [12]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_val = scaler.transform(X_val)

In [13]:
model = Sequential([
         Dense(16, activation='sigmoid', input_shape=(X_train.shape[1],), kernel_regularizer=regularizers.l2(0.05)),
         Dense(24, activation='sigmoid', kernel_regularizer=regularizers.l2(0.05)),
         Dense(10, activation='sigmoid', kernel_regularizer=regularizers.l2(0.05)),
         Dense(y.shape[1], activation='softmax')
     ])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [14]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [15]:
history = model.fit(X_train, y_train, epochs=260, batch_size=64, validation_data=(X_val, y_val))

Epoch 1/260
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - accuracy: 0.4064 - loss: 3.9019 - val_accuracy: 0.3875 - val_loss: 3.5695
Epoch 2/260
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4042 - loss: 3.4632 - val_accuracy: 0.3875 - val_loss: 3.2463
Epoch 3/260
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3818 - loss: 3.1719 - val_accuracy: 0.3875 - val_loss: 2.9662
Epoch 4/260
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3750 - loss: 2.9002 - val_accuracy: 0.3875 - val_loss: 2.7137
Epoch 5/260
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3990 - loss: 2.6449 - val_accuracy: 0.3875 - val_loss: 2.4878
Epoch 6/260
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3890 - loss: 2.4176 - val_accuracy: 0.3875 - val_loss: 2.2878
Epoch 7/260
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.3925 - loss: 2.2267 - val_accuracy: 0.3875 - val_loss: 2.1111
Epoch 8/260
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.3957 - loss: 2.0391 - val_accuracy: 0.3875 

In [16]:
model.save("wine_quality_model.h5")

In [17]:
train_loss, train_acc = model.evaluate(X_train, y_train)
test_loss, test_acc = model.evaluate(X_val, y_val)
print(f"Training Accuracy: {train_acc:.4f}")
print(f"Testing Accuracy: {test_acc:.4f}")

35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4677 - loss: 1.1916
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4051 - loss: 1.2399 
Training Accuracy: 0.4343
Testing Accuracy: 0.4208


In [18]:
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_test, axis=1)

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step


In [19]:
conf_matrix = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:\n", conf_matrix)

Confusion Matrix:
 [[  0   9   0   0   0]
 [  0  94   0   0   0]
 [  0 107   0   0   0]
 [  0  27   0   0   0]
 [  0   3   0   0   0]]


In [20]:
report = classification_report(y_true, y_pred)
print("Classification Report:\n", report)

Classification Report:
               precision    recall  f1-score   support

           1       0.00      0.00      0.00         9
           2       0.39      1.00      0.56        94
           3       0.00      0.00      0.00       107
           4       0.00      0.00      0.00        27
           5       0.00      0.00      0.00         3

    accuracy                           0.39       240
   macro avg       0.08      0.20      0.11       240
weighted avg       0.15      0.39      0.22       240



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [21]:
def predict_wine_quality(input_features):
    model = keras.models.load_model("wine_quality_model.h5")
    input_scaled = scaler.transform(np.array(input_features).reshape(1, -1))
    prediction = model.predict(input_scaled)
    predicted_class = encoder.inverse_transform(prediction.reshape(1, -1))
    return predicted_class[0][0]

In [22]:
sample_input = X_test[0].reshape(1, -1)
print("Predicted Quality:", predict_wine_quality(sample_input))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
Predicted Quality: 5


In [23]:
X_test[0]

array([-0.41418981,  0.83471189, -0.20598449, -0.17399159,  0.49422178,
        0.39728349,  1.12148244, -0.20549378, -0.67747976, -0.28356736,
       -1.05481025])

In [24]:
y[0]

array([0., 0., 1., 0., 0., 0.])